In [3]:
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()
import MySQLdb
import mysql.connector
from sqlalchemy import create_engine
import numpy as np
from tqdm import trange
import time
import string
import re
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import pymysql
pymysql.install_as_MySQLdb()
import pyodbc
import sqlalchemy
import MySQLdb
from sqlalchemy import create_engine
import mysql.connector
import pandas as pd
import xlrd
# from fuzzywuzzy import fuzz
import re
import wikipedia
from collections import Counter
from tqdm import tqdm
import time
import rapidfuzz
from fuzzywuzzy import process, fuzz

/home/jupyter/.local/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
username = 'swang'
password = 'prim-zorro-2410'
engine = create_engine('mysql://'+username+':'+password + '@localhost:3306')


mydb = mysql.connector.connect(
    host='localhost',
    user=username,
    passwd=password,
    database = 'colleges',
    auth_plugin='mysql_native_password')

mycursor = mydb.cursor()

In [85]:
query1 = '''select mykCollegeId, arcoName, city, state from colleges.schools'''
data = pd.read_sql(query1, engine)
schools_Id = data.copy()
schools_Id = schools_Id
schools_Id

,mykCollegeId,arcoName,city,state
0,1001,Auburn University,Auburn,AL
1,1002,Birmingham Southern College,Birmingham,AL
2,1003,Concordia College Alabama,Selma,AL
3,1004,CollegeAmerica-Flagstaff,Flagstaff,AZ
4,1005,Hendrix College,Conway,AR
...,...,...,...,...
6870,8743,University of the West,Rosemead,CA
6871,8744,Won Institute of Graduate Studies,Glenside,PA
6872,8745,American Islamic College,Chicago,IL
6873,8746,Respect Graduate School,Bethlehem,PA


In [111]:
wikipedia_table = pd.read_csv('WikipediaSchools-Copy1.csv')
wikipedia_table

,arcoName,city,state
0,Boston University,Boston,Massachusetts
1,Brandeis University,Waltham,Massachusetts
2,Brown University,Providence,Rhode Island
3,California Institute of Technology,Pasadena,California
4,Carnegie Mellon University,Pittsburgh,Pennsylvania
...,...,...,...
1777,Highline College,Des Moines,Washington
1778,Pierce College,Lakewood,Washington
1779,Wisconsin Technical College System,None,None
1780,Wisconsin Technical College System,None,None


In [7]:
wikipedia_table.dtypes

arcoName     object
city         object
state        object
schoolURL    object
dtype: object

# Data Clean Function

In [87]:
def drop_pun(str):
#     for i in item:
#         if i in string.punctuation:
#             item = re.sub(i,'',item)
#     return item
    str = re.sub(r'[^\w\s]','',str)
    str = re.sub(' ','',str)
    str = re.sub('university','1',str)
    str = re.sub('college','2',str)
    str = re.sub('the','3',str)
    return str

def to_lower(item):
    return item.lower()

#
def stringCleaning(str):
    if str is None:
        return None
    try:
        str = to_lower(str)
        str = drop_pun(str)
        
        return str
    except:
        return None

stringCleaning('ab.`college')

'ab2'

In [112]:
def DataClean(table, column1, column2, column3, newcolumn1, newcolumn2, newcolumn3):
    
    for i in range(len(table)):
        table.loc[i, newcolumn1] = stringCleaning(table[column1][i])
        table.loc[i, newcolumn2] = stringCleaning(table[column2][i])
        table.loc[i, newcolumn3] = stringCleaning(table[column3][i])


    
# DataClean(schools_Id, 'arcoName', 'city', 'state', 'schoolURL', 'Tb1_arcoName', 'Tb1_city', 'Tb1_state', 'Tb1_schoolURL')
# DataClean(schools_withoutId, 'arcoName', 'city', 'state', 'schoolURL', 'Tb2_arcoName', 'Tb2_city', 'Tb2_state', 'Tb2_schoolURL')

In [89]:
DataClean(schools_Id, 'arcoName', 'city', 'state', 'Tb1_arcoName', 'Tb1_city', 'Tb1_state')
schools_Id

,mykCollegeId,arcoName,city,state,Tb1_arcoName,Tb1_city,Tb1_state
0,1001,Auburn University,Auburn,AL,auburn1,auburn,al
1,1002,Birmingham Southern College,Birmingham,AL,birminghamsou3rn2,birmingham,al
2,1003,Concordia College Alabama,Selma,AL,concordia2alabama,selma,al
3,1004,CollegeAmerica-Flagstaff,Flagstaff,AZ,2americaflagstaff,flagstaff,az
4,1005,Hendrix College,Conway,AR,hendrix2,conway,ar
...,...,...,...,...,...,...,...
6870,8743,University of the West,Rosemead,CA,1of3west,rosemead,ca
6871,8744,Won Institute of Graduate Studies,Glenside,PA,woninstituteofgraduatestudies,glenside,pa
6872,8745,American Islamic College,Chicago,IL,americanislamic2,chicago,il
6873,8746,Respect Graduate School,Bethlehem,PA,respectgraduateschool,bethlehem,pa


In [103]:
wikipedia_table

,arcoName,city,state
0,Boston University,Boston,Massachusetts
1,Brandeis University,Waltham,Massachusetts
2,Brown University,Providence,Rhode Island
3,California Institute of Technology,Pasadena,California
4,Carnegie Mellon University,Pittsburgh,Pennsylvania
...,...,...,...
1777,Highline College,Des Moines,Washington
1778,Pierce College,Lakewood,Washington
1779,Wisconsin Technical College System,None,None
1780,Wisconsin Technical College System,None,None


In [113]:
DataClean(wikipedia_table, 'arcoName', 'city', 'state', 'Tb2_arcoName', 'Tb2_city', 'Tb2_state')
wikipedia_table

,arcoName,city,state,Tb2_arcoName,Tb2_city,Tb2_state
0,Boston University,Boston,Massachusetts,boston1,boston,massachusetts
1,Brandeis University,Waltham,Massachusetts,brandeis1,waltham,massachusetts
2,Brown University,Providence,Rhode Island,brown1,providence,rhodeisland
3,California Institute of Technology,Pasadena,California,californiainstituteoftechnology,pasadena,california
4,Carnegie Mellon University,Pittsburgh,Pennsylvania,carnegiemellon1,pittsburgh,pennsylvania
...,...,...,...,...,...,...
1777,Highline College,Des Moines,Washington,highline2,desmoines,washington
1778,Pierce College,Lakewood,Washington,pierce2,lakewood,washington
1779,Wisconsin Technical College System,None,None,wisconsintechnical2system,none,none
1780,Wisconsin Technical College System,None,None,wisconsintechnical2system,none,none


In [114]:
wikipedia_table = wikipedia_table.dropna()
wikipedia_table = wikipedia_table.drop(wikipedia_table[wikipedia_table.city == 'None'].index)
wikipedia_table = wikipedia_table.drop(wikipedia_table[wikipedia_table.state == 'None'].index)
wikipedia_table

,arcoName,city,state,Tb2_arcoName,Tb2_city,Tb2_state
0,Boston University,Boston,Massachusetts,boston1,boston,massachusetts
1,Brandeis University,Waltham,Massachusetts,brandeis1,waltham,massachusetts
2,Brown University,Providence,Rhode Island,brown1,providence,rhodeisland
3,California Institute of Technology,Pasadena,California,californiainstituteoftechnology,pasadena,california
4,Carnegie Mellon University,Pittsburgh,Pennsylvania,carnegiemellon1,pittsburgh,pennsylvania
...,...,...,...,...,...,...
1774,J. Sargeant Reynolds Community College,Richmond,Virginia,jsargeantreynoldscommunity2,richmond,virginia
1775,Paul D. Camp Community College,Franklin,Virginia,pauldcampcommunity2,franklin,virginia
1776,Clark College,Vancouver,Washington,clark2,vancouver,washington
1777,Highline College,Des Moines,Washington,highline2,desmoines,washington


In [115]:
wikipedia_table.to_csv('./temp.csv', index = False)

In [116]:
schools_Id

,mykCollegeId,arcoName,city,state,Tb1_arcoName,Tb1_city,Tb1_state
0,1001,Auburn University,Auburn,AL,auburn1,auburn,al
1,1002,Birmingham Southern College,Birmingham,AL,birminghamsou3rn2,birmingham,al
2,1003,Concordia College Alabama,Selma,AL,concordia2alabama,selma,al
3,1004,CollegeAmerica-Flagstaff,Flagstaff,AZ,2americaflagstaff,flagstaff,az
4,1005,Hendrix College,Conway,AR,hendrix2,conway,ar
...,...,...,...,...,...,...,...
6870,8743,University of the West,Rosemead,CA,1of3west,rosemead,ca
6871,8744,Won Institute of Graduate Studies,Glenside,PA,woninstituteofgraduatestudies,glenside,pa
6872,8745,American Islamic College,Chicago,IL,americanislamic2,chicago,il
6873,8746,Respect Graduate School,Bethlehem,PA,respectgraduateschool,bethlehem,pa


# FuzzyWuzzy

In [208]:
scorer_dict = { 'R':fuzz.ratio, 
                'PR': fuzz.partial_ratio, 
                'TSeR': fuzz.token_set_ratio, 
                'TSoR': fuzz.token_sort_ratio,
                'PTSeR': fuzz.partial_token_set_ratio, 
                'PTSoR': fuzz.partial_token_sort_ratio, 
                'WR': fuzz.WRatio, 
                'QR': fuzz.QRatio,
                'UWR': fuzz.UWRatio, 
                'UQR': fuzz.UQRatio }

https://medium.com/analytics-vidhya/matching-messy-pandas-columns-with-fuzzywuzzy-4adda6c7994f

In [117]:
# model1 : match schoolname
# model2: match schoolname, state, city
# model3: match schoolname state city, url

def matchHelper(tg1, tg2, tg3, cl1, cl2, cl3, table1, model='model2'):
    max_score = -1
    max_name = ''
    max_pk = ''
#     count = 0
    for i in range(len(table1)):
#         if count == 3:
#             break
#         count = count + 1
        pk = table1['mykCollegeId'][i]
        if model == 'model1':
            score1 = fuzz.token_set_ratio(tg1, table1[cl1][i]) 
            score = score1
        elif model == 'model2':
            score1 = fuzz.token_set_ratio(tg1, table1[cl1][i])
            score2 = fuzz.partial_ratio(tg2, table1[cl2][i]) 
            score3 = fuzz.partial_ratio(tg3, table1[cl3][i])
            if (score2 > 75 & score3 > 75):
                score = score1 * 50 / 100 + score2 * 25 / 100 + score3 * 25 / 100
            else:
                score = score1 * 20 / 100 + score2 * 40 / 100 + score3 * 40 / 100
        else:
            print('Wrong model!')
            return ('',0,'')
            
        #school name
#         score1 = fuzz.token_set_ratio(tg1, table1[cl1][i])
      
#         #city
#         score2 = fuzz.partial_ratio(tg2, table1[cl2][i])
        
# #         #state
#         score3 = fuzz.partial_ratio(tg3, table1[cl3][i])
        
# #         #url
#         score4 = fuzz.token_sort_ratio (tg4, table1[cl4][i])
#         print(str(score1) +'--------'+ tg1 +'-----' + table1[cl1][i])
#         print(str(score2) +'--------'+ tg2 +'-----' + table1[cl2][i])
#         print(str(score3) +'--------'+ tg3 +'-----' + table1[cl3][i])
#         print(str(score4) +'--------'+ tg4 +'-----' + table1[cl4][i])
#         print("-------------------------------------------------------------")

        
        
#         score = (score1 + score2 + score3 + score4) / 4
#         print('score: ' + str(score) + '/' + 'name_score1: ' + str(score1) + '/' + 'city_score2: ' + str(score2) + '/'
#              + 'state_score3: ' + str(score3) + '/')
        if (score > 75) & (score > max_score):
            max_name = table1[cl1][i]
            max_score = score
            max_pk = pk
            
    return (max_name, max_score, max_pk)

#max_name : table1 primary key.

In [118]:
# table1  old table in myklovr
# table2  new table waiting to match
# name_dict = {}
# fail_dict = {}

def FuzzyMatch(table1, table2, tb1_column1, tb1_column2, tb1_column3, tb2_column1, tb2_column2, tb2_column3, model):
    names = []
    res = []
    fails = []
#     count = 0
    for i in range(len(table2)):
#         if count == 2:
#             break
#         count = count + 1
        match = matchHelper(table2[tb2_column1][i], table2[tb2_column2][i], table2[tb2_column3][i], tb1_column1, tb1_column2, tb1_column3, table1, model)
#         print(match)
        if match[1] >= 80:
            name = (str(match[2]), str(match[0]) + ' | ' + str(table2[tb2_column1][i]) + ' | ' + str(match[1]))
            print(name)
            names.append(name)
            temp = (str(match[2]), str(table2[tb2_column1][i]))
            print(temp)
            res.append(temp)
        else:
            fail = (str(match[2]), str(match[0]) + ' | ' + str(table2[tb2_column1][i]) + ' | ' + str(match[1]))
#             print(fail)
            fails.append(fail)
    name_dict = dict(names)
    res_dict = dict(res)
    return (name_dict, res_dict)

# FuzzyMatch(schools_Id, schools_withoutId, 'Tb1_arcoName', 'Tb1_city', 'Tb1_state', 'Tb1_schoolURL', 'Tb2_arcoName', 'Tb2_city', 'Tb2_state', 'Tb2_schoolURL', 'model3')

In [119]:
res = FuzzyMatch(schools_Id, wikipedia_table, 'Tb1_arcoName', 'Tb1_city', 'Tb1_state', 'Tb2_arcoName', 'Tb2_city', 'Tb2_state', 'model2')



('8328', 'boston1 | boston1 | 100.0')
('8328', 'boston1')
('8329', 'brandeis1 | brandeis1 | 100.0')
('8329', 'brandeis1')
('2535', 'brown1 | brown1 | 80.0')
('2535', 'brown1')
('7010', 'californiainstituteoftechnology | californiainstituteoftechnology | 100.0')
('7010', 'californiainstituteoftechnology')
('2307', 'carnegiemellon1 | carnegiemellon1 | 80.0')
('2307', 'carnegiemellon1')
('1965', 'casewesternreserve1 | casewesternreserve1 | 100.0')
('1965', 'casewesternreserve1')
('1313', 'york2 | columbia1 | 82.8')
('1313', 'columbia1')
('1313', 'york2 | graduatecentercuny | 81.8')
('1313', 'graduatecentercuny')
('2325', 'drexel1 | drexel1 | 80.0')
('2325', 'drexel1')
('7611', 'emory1 | emory1 | 80.0')
('7611', 'emory1')
('4558', 'cortivainstitutegroton | georgewashington1 | 81.2')
('4558', 'georgewashington1')
('8360', 'harvard1 | harvard1 | 100.0')
('8360', 'harvard1')
('8274', 'johnshopkins1 | johnshopkins1 | 80.0')
('8274', 'johnshopkins1')


KeyError: 14

In [363]:
len(res[0])

0

In [268]:
res[0]

{'8328': 'bostonuniversity | bostonuniversity | 100.0',
 '8329': 'brandeisuniversity | brandeisuniversity | 100.0',
 '2535': 'brownuniversity | brownuniversity | 83.33333333333333',
 '7010': 'californiainstituteoftechnology | californiainstituteoftechnology | 100.0',
 '2307': 'carnegiemellonuniversity | carnegiemellonuniversity | 83.33333333333333',
 '1965': 'casewesternreserveuniversity | casewesternreserveuniversity | 100.0',
 '1313': 'yorkcollege | citycollegeofnewyork | 84.0',
 '2325': 'drexeluniversity | drexeluniversity | 83.33333333333333',
 '7611': 'emoryuniversity | emoryuniversity | 83.33333333333333',
 '7451': 'georgewashingtonuniversity | georgewashingtonuniversity | 83.33333333333333',
 '8360': 'harvarduniversity | harvarduniversity | 100.0',
 '8274': 'johnshopkinsuniversity | johnshopkinsuniversity | 83.33333333333333',
 '1656': 'newyorkuniversity | newyorkuniversity | 83.33333333333333',
 '8405': 'northeasternuniversity | northeasternuniversity | 100.0',
 '7809': 'northw